In [1]:
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
import os

from keras.utils import to_categorical
from sklearn import preprocessing

from PIL import Image
import itertools
from tqdm import tqdm

C:\Users\wogur\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
root = '../dataset/'
ct_set = os.path.join(root,'ct_train_test/ct_train/')
mr_set = os.path.join(root,'mr_train_test/mr_train/')

In [3]:
ct_list = os.listdir(ct_set)
ct_images = list()
ct_labels = list()
for ct_l in ct_list:
    if 'image' in ct_l:
        file_path = os.path.join(ct_set, ct_l)
        fn = os.listdir(file_path)
        ct_images.append(nib.load(file_path + '/' + fn[0]))
    elif 'label' in ct_l:
        file_path = os.path.join(ct_set, ct_l)
        fn = os.listdir(file_path)
        ct_labels.append(nib.load(file_path + '/' + fn[0]))

In [4]:
mr_list = os.listdir(mr_set)
mr_images = list()
mr_labels = list()
for mr_l in mr_list:
    if 'image' in mr_l:
        file_path = os.path.join(mr_set, mr_l)
        fn = os.listdir(file_path)
        mr_images.append(nib.load(file_path + '/' + fn[0]))
    elif 'label' in mr_l:
        file_path = os.path.join(mr_set, mr_l)
        fn = os.listdir(file_path)
        mr_labels.append(nib.load(file_path + '/' + fn[0]))

In [5]:
ct_cnt = len(ct_images)
mr_cnt = len(mr_images)

# image shape

In [6]:
ct_size = list()
mr_size = list()
for ct_image in ct_images:
    ct_size.append(ct_image.shape)    
for mr_image in mr_images:
    mr_size.append(mr_image.shape)

In [7]:
ct_size

[(512, 512, 363),
 (512, 512, 239),
 (512, 512, 298),
 (512, 512, 200),
 (512, 512, 177),
 (512, 512, 248),
 (512, 512, 243),
 (512, 512, 222),
 (512, 512, 293),
 (512, 512, 274),
 (512, 512, 239),
 (512, 512, 177),
 (512, 512, 211),
 (512, 512, 358),
 (512, 512, 300),
 (512, 512, 333),
 (512, 512, 283),
 (512, 512, 187),
 (512, 512, 297),
 (512, 512, 363)]

In [8]:
mr_size

[(512, 512, 160),
 (512, 512, 128),
 (288, 288, 160),
 (288, 288, 120),
 (288, 288, 130),
 (256, 256, 160),
 (288, 288, 180),
 (288, 288, 130),
 (512, 512, 120),
 (288, 288, 160),
 (288, 288, 160),
 (512, 512, 128),
 (512, 512, 112),
 (512, 512, 160),
 (340, 340, 200),
 (288, 288, 130),
 (288, 288, 140),
 (288, 288, 150),
 (288, 288, 135),
 (288, 288, 135)]

# image resize

In [9]:
def resizing(img, resize):
    bigsize = max(img.size)
    bg = Image.new(mode='L', size=(bigsize,bigsize), color=0)
    offset = (int(round(((bigsize - img.size[0])/2),0)), int(round(((bigsize - img.size[1])/2),0)))
    bg.paste(img,offset)
    bg = bg.resize(size=(resize,resize))
    return bg

In [10]:
def image_preprocess(images):
    '''
    preprocessing image : zero padding, rescale, onehotencoding
    :param images: Raw images
    :return: preprocessed images
    '''
    processed_imgs = np.zeros((len(images),256,256,256,1))
    for img in tqdm(images):
        resized_img = np.zeros((256, 256, img.shape[2]))
        pad_img = np.zeros((256, 256, 256))
        for i in range(img.shape[2]):
            im = Image.fromarray(img.get_data()[:, :, i])
            resized_img[:, :, i] = np.asarray(im.resize((256, 256)))

        resized_img[resized_img < 0] = 0

        for j in range(resized_img.shape[0]):
            im = Image.fromarray(resized_img[j, :, :])
            im = resizing(im, 256)
            pad_img[j, :, :] = np.asarray(im)

        pad_img = pad_img.reshape(1,256,256,256,1)
        pad_img = pad_img / 255.

        processed_imgs = np.vstack([processed_imgs, pad_img])

    return processed_imgs

In [11]:
def mask_preprocess(masks, class_num=7):
    class_num = class_num + 1 # background
    onehot_masks = np.zeros((len(masks),256,256,256,class_num))
    
    for mask in tqdm(masks):
        initial_data = mask.get_data()
        
        initial_size_x = initial_data.shape[0]
        initial_size_y = initial_data.shape[1]
        initial_size_z = initial_data.shape[2]

        new_size_x = 256
        new_size_y = 256
        new_size_z = 256

        delta_x = initial_size_x/new_size_x
        delta_y = initial_size_y/new_size_y
        delta_z = initial_size_z/new_size_z

        new_data = np.zeros((new_size_x,new_size_y,new_size_z))

        for x, y, z in itertools.product(range(new_size_x),
                                         range(new_size_y),
                                         range(new_size_z)):
            new_data[x][y][z] = initial_data[int(x*delta_x)][int(y*delta_y)][int(z*delta_z)]
            
        # encoder
        raw_shape = new_data.shape
        flatten_data = new_data.reshape(-1)
        label_encoder = preprocessing.LabelEncoder()
        label_data = label_encoder.fit_transform(flatten_data)
        onehot_data = to_categorical(label_data, class_num)
        # reshape to raw shape
        onehot_data = onehot_data.reshape((1,) + raw_shape + (class_num,))
        onehot_masks = np.vstack([onehot_masks, onehot_data])
    
    return onehot_masks

In [ ]:
ct_pad_images = image_preprocess(ct_images); del(ct_images)
ct_pad_labels = mask_preprocess(ct_labels); del(ct_labels)

  0%|                                                                                           | 0/20 [00:00<?, ?it/s]

In [ ]:
mr_pad_images = image_preprocess(mr_images); del(mr_images)
mr_pad_labels = mask_preprocess(mr_labels); del(mr_labels)

### 아래 이미지는 channel 차원을 추가하기 전 이미지

In [ ]:
f, ax = plt.subplots(5,8, figsize=(25,20))
img_idx = 0
pad_idx = 0
for i in range(5*8):
    if (i%8)%2==0:
        ax[i//8,i%8].imshow(ct_images[img_idx].get_data()[256,:,:])
        ax[i//8,i%8].axis('off')
        img_idx+=1
    else:
        mask = np.argmax(ct_pad_images[pad_idx], axis=-1)
        ax[i//8,i%8].imshow(mask[128,:,:])
        ax[i//8,i%8].axis('off')
        pad_idx+=1

In [ ]:
print(ct_pad_images[0].shape)
img = nib.Nifti1Image(ct_pad_images[0], affine=np.eye(4))
nib.save(img,'../dataset/ct_imgs.nii')

In [ ]:
print(mr_pad_images[0].shape)
img = nib.Nifti1Image(mr_pad_images[0], affine=np.eye(4))
nib.save(img,'../dataset/mr_imgs.nii')

In [ ]:
print(ct_pad_labels[0].shape)
img = nib.Nifti1Image(ct_pad_labels[0], affine=np.eye(4))
nib.save(img,'../dataset/ct_labels.nii')

In [ ]:
print(mr_pad_labels[0].shape)
img = nib.Nifti1Image(mr_pad_labels[0], affine=np.eye(4))
nib.save(img,'../dataset/mr_labels.nii')

# Test image

In [ ]:
ct_test_dir = './dataset/ct_train_test/ct_test/'
ct_test_list = os.listdir(ct_test_dir)
ct_test_pad_images = list()

mr_test_dir = './dataset/mr_train_test/mr_test/'
mr_test_list = os.listdir(mr_test_dir)
mr_test_pad_images = list()

In [ ]:
ct_test_images = list()
for fn in ct_test_list:
    img_dir = os.path.join(ct_test_dir,fn)
    img_fn = os.listdir(img_dir)[0]
    im = nib.load(os.path.join(img_dir, img_fn))
    ct_test_images.append(im)
    
mr_test_images = list()
for fn in mr_test_list:
    img_dir = os.path.join(mr_test_dir,fn)
    img_fn = os.listdir(img_dir)[0]
    im = nib.load(os.path.join(img_dir, img_fn))
    mr_test_images.append(im)

In [ ]:
print('='*100)
print('CT')
print('='*100)
for img in ct_test_images:
    print(img.shape)
    
print('='*100)
print('MR')
print('='*100)
for img in mr_test_images:
    print(img.shape)

In [ ]:
f, ax = plt.subplots(1,5,figsize=(30,10))
for i in range(5):
    ax[i].imshow(ct_test_images[i].get_data()[256,:,:])
    ax[i].axis('off')

In [ ]:
ct_test_pad_images = image_preprocess(ct_test_images)
mr_test_pad_images = mask_preprocess(mr_test_images)